## PreReqPlusFourDemo.ipynb

### Written by Taiob Ali

@sqlworldwide

## Prerequisite:

- Azure SQL Database with data to play with
    
    - Sample database (I am using a .bacpac file that I can share with you)
    - [Amazon Sales Dataset](https:\www.kaggle.com\datasets\karkavelrajaj\amazon-sales-dataset)
    - Server Name: ta-aidemosqlserver.database.windows.net
    - Database Name: testdata
- Tables outside the dbo schema did not work for me
    
- Create an [Azure AI Search](https:\learn.microsoft.com\en-us\azure\search\search-what-is-azure-search) Service where you import data for indexing name "ta-azureaisearch"
    
    - Create a data source pointing to do.Customer table named "ta-datasource-customer"
    - Create an index using the above data source named "ta-index-customer", name the indexer "ta-indexer-customer"
    - <span style="font-size: 14px;">Using&nbsp;</span>  'Import and vectorize data' option create an index using the above data source on dbo.walmartProducts table from the testdata database name 'ta-vector-walmartproducts'
- Create an [Azure OpenAI](https:\learn.microsoft.com\en-us\azure\ai-services\openai\) service where you deploy the model and interact with the indexed data name "ta-openai"
    
    - <span style="color: var(--vscode-foreground); background-color: rgba(127, 127, 127, 0.1);">Deploy base model gpt-4 name ta-model-gpt-4</span>
    - Deploy model text-embedding-ada-002 name 'ta-model-text-embedding-ada-002'

## Demo1: [Natural language to SQL in the Azure portal query editor (preview)](https:\learn.microsoft.com\en-us\azure\azure-sql\copilot\query-editor-natural-language-to-sql-copilot?view=azuresql)

### Click Qery editor (preview) from the menu, login with Entra Admin or SQL authentication

### Question:

- Find customers and which products they have purchased for our mountain bikes by state and by total sales
    - Does a good job
- Count all customers, group by state, and give the top ten states with the most and least customers order by customer count
    - I get a mixed answer. Multiple selections on the same CTE are not allowed, but sometimes, it gives separate CTEs for each select statement.
- Find the hierarchy of product categories at any level
    - Honest but interesting answer
- Show me which tables have consumed the most space in this database
    - Failed few days ago(most likely no access to system tables; we will come back to this in the following demo), and now I get a perfect query
- Create a stored procedure that shows all sales by customer ID, with Customer ID as the input parameter, and raises an error if the customer ID does not exist. Drop the stored procedure if it already exists.
    - Create the stored procedure and run the below code to test

In [ ]:
EXEC [dbo].[ShowSalesByCustomerID]
		@CustomerID= 29847
GO

EXEC [dbo].[ShowSalesByCustomerID]
		@CustomerID= 8
GO

/*
DROP PROCEDURE IF EXISTS [dbo].[ShowSalesByCustomerID];
GO
*/


## Demo2: [Microsoft Copilot skills in Azure SQL Database (preview)](https:\learn.microsoft.com\en-us\azure\azure-sql\copilot\copilot-azure-sql-overview?view=azuresql)

### Question:

- <span style="font-size: 14px;">Show me which tables have consumed the most space in this database</span>
- Show me the top preemptive waits for this database
- Is my database experiencing high I/O?
- <span style="font-size: 14px;">Can you show me the current I/O metrics for my database?</span>

## Demo3: Keyword search with Large Language Model (LLM)

- Using Azure Ai Seach service named "ta-openai"
    
    - Create an index on dbo.customer table from the testdata database name ta-index-customer
- Deploy model GPT-4 
    
    - Deploy base model gpt-4 name ta-model-gpt-4
    - Add data ta-index-customer, Search type Keyword, authentication API Key, 
    - Give the model instructions and context: You are an AI assistant who helps company executives gain customer insight. Please provide as many details as possible about each customer, such as first name, last name, and which company they represent. Whenever you respond, please format your answer to make it very readable, including bullet points.
- Change 'Tokens per Minute Rate Limit to 20k (Click deployments --\>edit)
    
- Question:
    
    - What customer names work for the company, ' Bikes and Motorbikes'?
    - Which company name has the most number of customers? (if not getting an answer, tweak the 'Retrieved documents' setting)
    - Which customer uses the phone number 783-555-0110?

In [ ]:
/*
Verify the answer of question: What customer names work for the company,' Bikes and Motorbikes'?
*/
SELECT 
	FirstName,
	LastName,
	CompanyName
FROM [dbo].customer
WHERE CompanyName ='Bikes and Motorbikes'

In [3]:
/*
Verify the answer of question: Which company name has the most number of customers?
*/
SELECT
	companyname, 
	COUNT(0) AS countOfCustomersPerCompany
FROM dbo.customer
GROUP BY companyname
ORDER BY countOfCustomersPerCompany DESC, companyname

SELECT 
	* 
FROM DBO.customer 
WHERE companyname ='Locks Company'

In [ ]:
/*
Verify the answer of question: Which customer uses the phone number 783-555-0110?
*/
SELECT 
	FirstName,
	LastName,
	Phone
FROM [dbo].customer
WHERE Phone ='783-555-0110'

## Demo4: [Integrated Vectorization](https:\techcommunity.microsoft.com\blog\azure-ai-services-blog\announcing-the-public-preview-of-integrated-vectorization-in-azure-ai-search\3960809), Semantic ranker with Azure SQL DatabaseDeploy model text-embedding-ada-002 with vector search

- Deploy model text-embedding-ada-002 name 'ta-model-text-embedding-ada-002'
    
- Using AI Azure Seach service
    
    - Create an index on dbo.walmartProducts table from the testdata database name 'ta-vector-walmartproducts'
    - Use 'Import and vectorize data' option
    - Search for 'Curtain' and 'Furniture' using built-in [Search Explorer](https:\learn.microsoft.com\en-us\azure\search\search-explorer)
        - Query options turn on 'Hide vector values in search results.'
    - Do a search with JSON, which allows parameterized queries.
    
    ```
    {
    "search": "upscale sandal",
    "count": true,
    "select": "brand, review_count, product_id, product_name, discount",
    "top": 100
     }
    
    ```
    
- Question in Azure OpenAI service (using Azure AI studio)
    
    - <span style="background-color: rgba(127, 127, 127, 0.1); color: var(--vscode-foreground);">I am on a tight budget</span><span style="font-size: 14px;">, but I need inexpensive furniture options that are easy to install and make my living room more appealing</span>.
- Give the mode instructions and ask the same questions (see the difference in quality of the answer)
    
    - You are an AI assistant who helps people shop for products. Give as many details as possible about each product, such as price. Whenever you respond, please format your answer to make it very readable, including bullet points